### Lung Health Detector

This project uses a convolutional neural network to classify chest x-rays into "normal" and "pneumonia".  
Dataset available at: https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia

#### Imports

In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

In [2]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

Using TensorFlow backend.


In [3]:
training_set = train_datagen.flow_from_directory('chest_xray/train', 
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')
test_set = train_datagen.flow_from_directory('chest_xray/test', 
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')
val_set = train_datagen.flow_from_directory('chest_xray/val', 
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


#### Creating and training the classifier

In [4]:
classifier = keras.Sequential([
    keras.layers.Convolution2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(activation='relu', units=128),
    keras.layers.Dense(activation='sigmoid', units=1)])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [6]:
from IPython.display import display 
from PIL import Image

classifier.fit_generator(training_set,
                        steps_per_epoch=1000,
                        epochs=10,
                        validation_data=val_set,
                        validation_steps=800)

Epoch 1/10
1000/1000 [==============================] - 960s 960ms/step - loss: 0.2250 - acc: 0.9080 - val_loss: 0.8028 - val_acc: 0.6759
Epoch 2/10
1000/1000 [==============================] - 839s 839ms/step - loss: 0.1450 - acc: 0.9438 - val_loss: 0.9771 - val_acc: 0.6475
Epoch 3/10
1000/1000 [==============================] - 765s 765ms/step - loss: 0.1264 - acc: 0.9521 - val_loss: 0.7571 - val_acc: 0.6859
Epoch 4/10
1000/1000 [==============================] - 871s 871ms/step - loss: 0.1182 - acc: 0.9541 - val_loss: 0.6848 - val_acc: 0.7195
Epoch 5/10
1000/1000 [==============================] - 908s 908ms/step - loss: 0.1087 - acc: 0.9589 - val_loss: 1.0317 - val_acc: 0.6696
Epoch 6/10
1000/1000 [==============================] - 806s 806ms/step - loss: 0.0961 - acc: 0.9624 - val_loss: 0.5264 - val_acc: 0.7667
Epoch 7/10
1000/1000 [==============================] - 684s 684ms/step - loss: 0.0880 - acc: 0.9671 - val_loss: 0.7041 - val_acc: 0.7120
Epoch 8/10
1000/1000 [============

#### Obtaining the accuracy

In [7]:
prediction = classifier.fit_generator(test_set, steps_per_epoch=200, epochs=10)

Epoch 1/10
200/200 [==============================] - 83s 413ms/step - loss: 0.2221 - acc: 0.9082
Epoch 2/10
200/200 [==============================] - 75s 375ms/step - loss: 0.1566 - acc: 0.9337
Epoch 3/10
200/200 [==============================] - 74s 371ms/step - loss: 0.1417 - acc: 0.9420
Epoch 4/10
200/200 [==============================] - 76s 378ms/step - loss: 0.1254 - acc: 0.9537
Epoch 5/10
200/200 [==============================] - 78s 391ms/step - loss: 0.1055 - acc: 0.9607
Epoch 6/10
200/200 [==============================] - 79s 395ms/step - loss: 0.0847 - acc: 0.9686
Epoch 7/10
200/200 [==============================] - 77s 385ms/step - loss: 0.0835 - acc: 0.9662
Epoch 8/10
200/200 [==============================] - 86s 429ms/step - loss: 0.0680 - acc: 0.9747
Epoch 9/10
200/200 [==============================] - 90s 451ms/step - loss: 0.0598 - acc: 0.9779
Epoch 10/10
200/200 [==============================] - 91s 453ms/step - loss: 0.0646 - acc: 0.9763


In [8]:
score = classifier.evaluate(test_set, steps=500)

500/500 [==============================] - 205s 409ms/step - loss: 0.0663 - acc: 0.9756


In [10]:
score[1]

0.975641

In [11]:
from keras.models import load_model

classifier.save('lungs_CNN.h5')